In [ ]:
__nbid__ = '0075'
__author__  = 'Brian Merino <brian.merino@noirlab.edu>, David Herrera <david.herrera@noirlab.edu>'
__version__ = '20251215' # aaaammdd; marca de fecha de la versión de este notebook
__keywords__ = ['gnirs','gemini','star','dragons']

# Reducción de fuente puntual de rendija larga en banda L de Gemini usando DRAGONS Python API
#### (_Traducción de [GNIRS_longslit_Bestar.ipynb](https://github.com/astro-datalab/notebooks-latest/blob/master/04_HowTos/DataReduction/DRAGONS_reduction_examples/GNIRS_longslit_Bestar/GNIRS_longslit_Bestar.ipynb)_)
***
## Datos públicos de archivo de GN-2017B-Q-81 (HD41335)
#### adaptado de https://gnirsls-drtutorial.readthedocs.io/en/v4.0.0/ex3_gnirsls_Lband10mm_api.html
***

## Tabla de contenido
* [Objetivos](#goals)
* [Resumen](#summary)
* [Avisos legales y atribuciones](#disclaimer)
* [Importaciones y configuración](#imports)
* [Preparar el directorio de trabajo](#Prepare)
* [Acerca del conjunto de datos](#About)
* [Descarga de datos para reducción](#Downloading_Data)
* [Configurar el registrador DRAGONS](#DRAGONS_logger)
* [Máscara de píxeles malos](#BPM)
* [Crear campo flat maestro](#Master_Flat)
* [Arco procesado - Solución de longitud de onda](#Arc)
* [Estándar Telúrico](#Telluric)
* [Observaciones científicas](#Science)
* [Mostrar espectro 1D](#Display)


<a class="anchor" id="goals"></a>
# Objetivos
Mostrar cómo reducir los datos de origen puntual de rendija larga de banda L de GNIRS usando el paquete Gemini DRAGONS en la plataforma científica Data Lab usando un kernel DRAGONS personalizado `"DRAGONS-4.0.0 (DL,Py3.12)``. Los pasos incluyen descargar datos del archivo Gemini, configurar un servicio de calibración DRAGONS, procesar flats, oscuros, una máscara de píxeles defectuosa y marcos científicos, y mostrar el espectro 1D reducido.

<a class="anchor" id="summary"></a>
# Resumen
DRAGONS es una plataforma de reducción de datos astronómicos basada en Python escrita por el Departamento de soporte al usuario de Gemini Science. Actualmente se puede utilizar para reducir datos de imágenes de los instrumentos Gemini GMOS, NIRI, Flamingos 2, GSAOI y GNIRS, así como datos espectroscópicos tomados con GNIRS, GHOST y GMOS en modo de rendija larga. En <a href="https://gnirsls-drtutorial.readthedocs.io/en/v4.0.0/index.html">aquí</a> hay una lista general de guías, manuales y tutoriales sobre el uso de DRAGONES.

El kernel DRAGONS está disponible en el entorno Data Lab, lo que permite a los usuarios acceder a las rutinas sin depender de la instalación del software en sus máquinas locales.

En este notebook, presentamos un ejemplo de un notebook DRAGONS Jupyter que funciona en el entorno de Data Lab para reducir los datos de banda L de Gemini North GNIRS de ejemplo. Este notebook no presentará todos los detalles de las muchas opciones disponibles para ajustar u optimizar el proceso de reducción de datos de DRAGONS GNIRS, incluidas las funciones interactivas; más bien, solo mostrará un ejemplo de reducción estándar.

Los datos utilizados en este ejemplo de notebook son espectroscopia de fuente puntual de rendija larga de banda L GNIRS del archivo Gemini de Be-star HD41335 del programa Gemini North "<a href="https://archive.gemini.edu/programinfo/GN-2017B-Q-81">Estructura y propiedades físicas de las envolturas circumestelares de las estrellas Be</a>", IP: Yanina Cochetti, ID del programa GN-2017B-Q-81.


<a class="anchor" id="attribution"></a>
# Avisos Legales y atribuciones

Avisos Legales
-----------
Tome en cuenta que usar el Astro Data Lab constituye un acuerdo con nuestros [Avisos Legales](https://datalab.noirlab.edu/disclaimers.php) mínimos.

Reconocimientos
---------------
Si ud. usa el **Astro Data Lab** en sus publicaciones de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios de datos proveeidos por el Astro Data Lab, el cual es parte del Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) del NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc."(Asociación de Universidaddes para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

Si utiliza **SPARCL junto con la plataforma de Astro Data Lab** (por medio de JupyterLab, línea de comando o interfaz de la web) en su publicación de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios o datos proporcionados por el "SPectra Analysis and Retrievable Catalog Lab" (SPARCL) (Laboratorio de Análisis y Catálogo Recuperable de Espectros) y el Astro Data Lab, ambos pertenecientes al Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) de NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc." (Asociación de Universidades para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

En cualquiera de los casos, **por favor cite las siguientes publicaciones**:

* Publicación del concepto de Data Lab: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del Astro Data Lab: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411.

Si hace referencia al Jupyterlab / Jupyter notebooks de Data Lab, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097.

Si publica en una revista de la AAS, agregue también la palabra clave `\facility{Astro Data Lab}`

Y si está usando SPARCL, por vor agregue también `\software{SPARCL}` y cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576.

La biblioteca de NOIRLab mantiene [listas de reconocimientos apropiados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar cuando se hacen publicaciones utilizando los recursos, servicios o datos del Laboratorio.

Para este notebook en particular, por favor también incluya el siguiente reconocimiento:

* Publicación de **DRAGONS**: Labrie et al., <a href="https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract">"DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South"</a>, ASPC, 523, 321L

* <a href="https://zenodo.org/record/7776065#.ZDg5qOzMLUI">Publicación del software **DRAGONS** de código abierto</a>

---- **Versión en Inglés** ----


# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

For this notebook specifically, please acknowledge:
* DRAGONS publication: Labrie et al., [DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South](https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract), ASPC, 523, 321L 

* [DRAGONS open source software publication](https://zenodo.org/record/7776065#.ZDg5qOzMLUI)


<a class="anchor" id="imports"></a>
# Importando bibliotecas de Python

In [ ]:
import warnings
import glob
import os
import time
import numpy as np
import astrodata
import shutil
import matplotlib.pyplot as plt

import gemini_instruments
from gempy.adlibrary import plotting
from gempy.utils import logutils
from gempy.adlibrary import dataselect

from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce

from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.filterwarnings("ignore")

<a class="anchor" id="Prepare"></a>
# Preparar el directorio de trabajo

Si tiene archivos intermedios que se crearon al ejecutar este código en el pasado, deberá eliminarlos de su directorio de trabajo. La siguiente celda define una función de limpieza que eliminará todos los archivos fits de su directorio de trabajo. Esta función se llamará nuevamente al final del tutorial, dejándolo solo con el producto final. De forma predeterminada, esta función eliminará todos los archivos en el directorio de trabajo. Si hay archivos que se han reducido previamente y que le gustaría conservar, configure `save_reduced=1` al llamar a la función.

In [ ]:
def clean_up(save_reduced=0):
    # ¿Ya existe el directorio de calibraciones?
    caldb_Exist = os.path.exists('./calibrations') 
    
    if caldb_Exist:
        shutil.rmtree('./calibrations', ignore_errors=True)

    # Eliminar los archivos de registro y lista existentes.
    work_dir_path = os.getcwd()
    work_dir = os.listdir(work_dir_path)

    for item in work_dir:
        if item.endswith(".log") or item.endswith(".list") or item.endswith(".lis"):
            os.remove(os.path.join(work_dir_path, item))
    
    # A continuación, eliminaremos todos los archivos fits existentes, excepto los archivos previamente reducidos, dependiendo de cómo haya configurado save_reduced.
    if save_reduced:
        all_files_0 = glob.glob('*.fits')
        save = dataselect.select_data(all_files_0, [], ['PROCESSED'])
        
        for s in save:
            os.remove(os.path.join(work_dir_path,s))

    else:
        all_files_0 = glob.glob('*.fits')
        for a in all_files_0:
            os.remove(os.path.join(work_dir_path,a))

        # El resultado final de este notebook incluye un PDF y archivos DAT. Entonces, si configura save_reduced=0, también debemos eliminar estos archivos.
        for item in work_dir:
            if item.endswith(".pdf") or item.endswith(".dat"):
                os.remove(os.path.join(work_dir_path, item))

In [ ]:
clean_up(save_reduced=0)

### <a class="anchor" id="About"></a>
# Acerca del conjunto de datos

Los datos son una observación de rendija larga de HD41335 en banda L con GNIRS. El programa de observación utilizó una rejilla de 10 l/mm con la cámara roja larga y una rendija de 0,1 segundos de arco, centrada en 3,7 μm.

Las calibraciones que utilizamos para este ejemplo son:
- BPM. Las máscaras de píxeles malos ahora se encuentran en Gemini Science Archive en lugar de estar empaquetadas con el software. Están asociados como las demás calibraciones. (La fecha en el nombre es la fecha “válida desde”).
- Flats tomados en la misma configuración que la ciencia. Se obtuvieron por la noche, inmediatamente después de las observaciones científicas.
- Arcos. No se obtienen arcos para datos de banda L. La calibración de la longitud de onda se realiza utilizando las líneas del cielo en la observación científica.
- Una observación de estándar telúrica tomada en la misma configuración que la científica y obtenida de noche, en este caso, justo antes de la secuencia de observación científica, y con una masa de aire similar.

| Tipo de observación | Nombre(s) de archivo |
| :--- | :--- |
| Ciencia | N20180114S0121-124 |
| Flats de ciencia | N20180114S0125-132 |
| Telúricas | N20180114S0113-116 |
| BPM | bpm_20121101_gnirs_gnirsn_11_full_1amp.fits |

<a class="anchor" id="Downloading_Data"></a>
# Descargando los datos

Descarga de datos de rendija larga del archivo Gemini al directorio de trabajo. Este paso sólo debe ejecutarse una vez.

Si ejecuta este notebook por primera vez y necesita descargar el conjunto de datos, configure la variable "download=True". El notebook no volverá a descargar el conjunto de datos si está configurado en False. Esto resultará especialmente útil si ejecuta los notebooks más de una vez.

In [ ]:
%%bash 

# Cree una lista de archivos FITS para descargar.
echo "\
http://archive.gemini.edu/file/N20180114S0121.fits
http://archive.gemini.edu/file/N20180114S0122.fits
http://archive.gemini.edu/file/N20180114S0123.fits
http://archive.gemini.edu/file/N20180114S0124.fits
http://archive.gemini.edu/file/N20180114S0125.fits
http://archive.gemini.edu/file/N20180114S0126.fits
http://archive.gemini.edu/file/N20180114S0127.fits
http://archive.gemini.edu/file/N20180114S0128.fits
http://archive.gemini.edu/file/N20180114S0129.fits
http://archive.gemini.edu/file/N20180114S0130.fits
http://archive.gemini.edu/file/N20180114S0131.fits
http://archive.gemini.edu/file/N20180114S0132.fits
http://archive.gemini.edu/file/N20180114S0113.fits
http://archive.gemini.edu/file/N20180114S0114.fits
http://archive.gemini.edu/file/N20180114S0115.fits
http://archive.gemini.edu/file/N20180114S0116.fits
http://archive.gemini.edu/file/bpm_20121101_gnirs_gnirsn_11_full_1amp.fits\
" > gnirs_Lband.list

In [ ]:
%%bash

download="True"

if [ $download == "True" ]; then
    wget --no-check-certificate -N -q -i gnirs_Lband.list

else
    echo "Skipping download. To download the data set used in this notebook, set download=True."
fi

<a class="anchor" id="DRAGONS_logger"></a>
# Configurando el registrador DRAGONS

DRAGONS viene con un administrador de calibración local que utiliza las mismas reglas de asociación de calibración que el Archivo del Observatorio Gemini. Esto permite que Reduce realice solicitudes a una base de datos local liviana para hacer coincidir las calibraciones procesadas cuando sea necesario para reducir un conjunto de datos.

Esto le indica al sistema dónde colocar la base de datos de calibración. Esta base de datos realizará un seguimiento de las calibraciones procesadas que le enviaremos.

In [ ]:
logutils.config(file_name='gnirs_ex3_tutorial.log')

caldb = cal_service.set_local_database()
caldb.init("w")

Cree una lista de todos los archivos FITS en el directorio

In [ ]:
all_files = glob.glob('*.fits')
print(all_files)

## **Una lista de los flats**

Los flats deberán apilarse juntos, así que use la selección de datos para identificar los archivos flats que son compatibles entre sí.

In [ ]:
flats = dataselect.select_data(all_files, ['FLAT'])

## **Una lista para los telúricos**

Actualmente, DRAGONS no tiene forma de diferenciar las estrellas telúricas del resto de datos. Esto se debe a que los encabezados GNIRS no contienen etiquetas que identifiquen específicamente a los telúricos. En la mayoría de los casos, especificar el descriptor "observation_class" y rechazar archivos con la etiqueta "CAL" son suficientes para distinguir entre observaciones telúricas y científicas.

In [ ]:
tellurics = dataselect.select_data(
    all_files,
    [],
    ['CAL'],
    dataselect.expr_parser('observation_class=="partnerCal"')
)

## **Una lista para la ciencia**

Para crear una lista para las observaciones científicas, tendremos archivos de selección de datos con la clase de observación 'science'. Nota: En este ejemplo, todas las observaciones científicas son de un único objetivo. Si el programa de observación tuviera múltiples objetivos, sería necesario dividirlos en sus propias listas.

In [ ]:
scitarget = dataselect.select_data(
    all_files,
    [],
    ['CAL'],
    dataselect.expr_parser('observation_class=="science" and object=="HD41335"')
)

<a class="anchor" id="BPM"></a>
## **Máscara de píxeles malos**

A partir de DRAGONS v3.1, las máscaras estáticas de píxeles malos (BPM) ahora se manejan como calibraciones. Se pueden descargar desde el archivo en lugar de venir empaquetados con el software. Se asocian automáticamente como cualquier otra calibración. Esto significa que el usuario ahora debe descargar los BPM junto con las otras calibraciones y agregar los BPM al administrador de calibración local.

In [ ]:
for bpm in dataselect.select_data(all_files, ['BPM']):
    caldb.add_cal(bpm)

<a class="anchor" id="Master_Flat"></a>
# Crear campo flat maestro

Los campos flats de rendija larga del GNIRS normalmente se obtienen de noche junto con la secuencia de observación para que coincida con la flexión del telescopio y del instrumento. El campo flat de rendija larga GNIRS solo requiere flats con lámparas. Restar los darks sólo aumenta el ruido. Los flats estarán apilados.

Después de ejecutar la siguiente celda, se creará un nuevo archivo llamado N20180114S0125_flat.fits.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(flats)
reduce_flats.runr()

<a class="anchor" id="Arc"></a>
# Arco procesado - solución de longitud de onda
La solución de longitud de onda para los datos de banda L se deriva de las longitudes de onda de máximos fuertes en el espectro de emisión del cielo. La calidad de la solución de longitud de onda depende del ancho y la fuerza de las características telúricas.

Después de ejecutar esta celda, se crearán tres archivos nuevos: atran_linelist_mk_3135-4266_wv5_r1940.dat, N20180114S0121_arc.fits y N20180114S0121_wavelengthSolutionDetermined.pdf

In [ ]:
reduce_sky = Reduce()
reduce_sky.files.extend(scitarget)
reduce_sky.recipename = 'makeWavecalFromSkyEmission'
reduce_sky.uparms = dict([('interactive', False)])
reduce_sky.runr()

<a class="anchor" id="Telluric"></a>
# Estándar Telúrico

El estándar telúrico observado antes de la observación científica es “hip 28910”. El tipo espectral de la estrella es A0V.

Para calcular y ajustar correctamente un modelo telúrico a la estrella, necesitamos conocer su temperatura efectiva. Para escalar adecuadamente la función de sensibilidad (para usar la estrella como estándar espectrofotométrico), necesitamos conocer la magnitud de la estrella. Esas son entradas a la primitiva fitTelluric.

La temperatura efectiva predeterminada de 9650 K es típica de una estrella A0V, que es el tipo espectral más común utilizado como estándar telúrico. Diferentes fuentes dan valores entre 9500 K y 9750 K y, por ejemplo, la lista de Eric Mamajek "[A Modern Mean Dwarf Stellar Color and Effective Temperature Sequence](https://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt)" cita la temperatura efectiva de una estrella A0V como 9700 K. El valor preciso tiene sólo un pequeño efecto sobre la sensibilidad derivada y aún menos efecto sobre la corrección telúrica, por lo que se puede utilizar la temperatura de cualquier fuente confiable. Usando Simbad, encontramos que la estrella tiene una magnitud de K=4,523, que es la banda de ondas más cercana a nuestra observación.

Tenga en cuenta que Astrodata reconoce los datos como espectros científicos de rendija larga GNIRS normales. Para calcular la corrección telúrica, debemos especificar la receta telúrica (reduceTelluric), de lo contrario, se ejecutará la reducción científica predeterminada.

Al ejecutar esta celda se creará un nuevo archivo llamado N20180114S0113_telluric.fits.

In [ ]:
reduce_telluric = Reduce()
reduce_telluric.files.extend(tellurics)
reduce_telluric.recipename = 'reduceTelluric'
reduce_telluric.uparms = dict([
            ('fitTelluric:bbtemp', 9700),
            ('fitTelluric:magnitude', 'K=4.523'),
            ('fitTelluric:interactive', False),
            ])
reduce_telluric.runr()

<a class="anchor" id="Science"></a>
# Observaciones científicas

El objeto de ciencia, HD41335, es una estrella Be. La secuencia de observación es un patrón de desplazamiento de oscilación ABBA. DRAGONS realizará operaciones de campo flat, calibrará longitudes de onda, restará el cielo, apilará los espectros alineados, extraerá la fuente, eliminará características telúricas y calibrará el flujo.

Con todas las calibraciones en el administrador de calibración local, solo necesita ejecutar la siguiente celda para reducir las observaciones científicas y extraer el espectro 1-D.

Cuando esta celda termine de ejecutarse, se crearán dos archivos nuevos llamados N20180114S0121_1D.fits y N20180114S0121_2D.fits.

In [ ]:
reduce_science = Reduce()
reduce_science.files.extend(scitarget)
reduce_science.runr()

<a class="anchor" id="Display"></a>
# Mostrar espectro 1D

Finalmente, puede utilizar _astrodata_ para mostrar el espectro reducido.

In [ ]:
ad = astrodata.open(reduce_science.output_filenames[0])
plotting.dgsplot_matplotlib(ad, 1)

## Opcional: limpiar el directorio de trabajo. (descomentar antes de ejecutar)

In [ ]:
# clean_up(save_reduced=1)